<a href="https://colab.research.google.com/github/herjh0405/DACON_Meal/blob/master/%EA%B5%AC%EB%82%B4%EC%8B%9D%EB%8B%B9_%EC%8B%9D%EC%88%98_%EC%9D%B8%EC%9B%90_%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DACON 한국토지주택공사
구내식당 식수 인원 예측 AI 경진대회

## 필요 Model Install

In [ ]:
!pip install pycaret
!pip install konlpy
# !pip install matplotlib -U

# # 한글 폰트 사용
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf

## Google Drive Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Import Library

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
np.random.seed(0)

from pycaret.regression import *
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, log_loss

from tqdm.notebook import tqdm
from konlpy.tag import Kkma

import torch
import os, re
# import matplotlib.pyplot as plt

# plt.rcParams['font.family']=['NanumGothic', 'sans-serif']
# plt.rcParams['axes.unicode_minus'] = False
# %matplotlib inline

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import os

def change_matplotlib_font(font_download_url):
    FONT_PATH = 'MY_FONT'
    
    font_download_cmd = f"wget {font_download_url} -O {FONT_PATH}.zip"
    unzip_cmd = f"unzip -o {FONT_PATH}.zip -d {FONT_PATH}"
    os.system(font_download_cmd)
    os.system(unzip_cmd)
    
    font_files = fm.findSystemFonts(fontpaths=FONT_PATH)
    for font_file in font_files:
        fm.fontManager.addfont(font_file)

    font_name = fm.FontProperties(fname=font_files[0]).get_name()
    matplotlib.rc('font', family=font_name)
    print("font family: ", plt.rcParams['font.family'])

font_download_url = "https://fonts.google.com/download?family=Noto%20Sans%20KR"
change_matplotlib_font(font_download_url)

## Load Data

In [ ]:
path = '/content/drive/MyDrive/DACON/Dacon_Industry_Meal/'
train = pd.read_csv(path+'train.csv')
test = pd.read_csv(path+'train.csv')

train.columns = ['일자', '요일', '정원','휴가자', '출장자', '시간외근무승인',\
                 '재택근무자', '조식', '중식', '석식', '중식계', '석식계']
test.columns = ['일자', '요일', '정원','휴가자', '출장자', '시간외근무승인',\
                 '재택근무자', '조식', '중식', '석식', '중식계', '석식계']

In [ ]:
train[['재택근무자', '중식계', '석식계']] = train[['재택근무자', '중식계', '석식계']].astype('int')
test[['재택근무자', '중식계', '석식계']] = test[['재택근무자', '중식계', '석식계']].astype('int')

train['일자'] = pd.to_datetime(train['일자'])
test['일자'] = pd.to_datetime(test['일자'])

## 데이터 시각화
* 시간에 따른 time_col의 변화를 관찰
    * time_col : `'정원', '휴가자', '출장자', '시간외근무승인', '재택근무자', '중식계', '석식계'`

* 삼성 SDS에서 실시한 적이 있는 대회라고 함
    * RNN에서 휴가와 날씨에 가중치를 둬서 좋은 결과를 냈다고 함 -> 참고하면 좋을 듯
    * 일기예보 데이터를 사용해보는 건 어떨까?

<br>

### 가설 1. 재택근무는 코로나 때문에 생겼을 것이다.
* 코로나 이전과 이후로, 식수 인원이 큰 차이가 있을 것이다

<br>

### 가설 2. 날씨가 큰 영향을 미칠것이다
* 월별, 계절별 데이터가 차이날 것

<br>

### 가설 3. 공휴일들과 휴가자는 큰 연관성이 있을 것이다.
* 공휴일과 그렇지 않은 날을 구분해보자.

<br>

### 궁금증 1. 조식이 과연 의미가 있을까?
* 일단 없을거라 생각되지만 확인하기 어려움


메뉴에 따라서 안먹기도 한다  
출장은 국내출장과 근무지내출장이 있는데 국내출장은 하루종일 안먹는다  
근무지내출장은 먹을수도 안먹을수도 있음  
휴가는 시간제휴가도 있는데 그거는 따로 분류가 안되어있다  
당일에도 휴가 가능하다.  
그냥 금요일날 휴가 존나 감  
공휴일이랑 연계해서 많이 감  

In [ ]:
time_col = list(train.columns[2:7]) + list(train.columns[-2:])

In [ ]:
plt.figure(figsize=(10, 8))
for col in time_col : 
    plt.plot(train['일자'], train[col], label=col)

plt.xlabel('날짜(일)', fontsize=10)
plt.ylabel('인원 수', fontsize=10)
plt.title('날짜 별 인원 수', fontsize=15)
plt.legend('best', labels=time_col)
plt.show()

### 요일 별 야근인구
* 수요일, 금요일에는 야근 인원이 현저히 적음을 확인
    * 석식계와 큰 연관이 있을 듯
    * 월화목 vs 수금으로 구분

In [ ]:
bar_width = 0.35
label = ['금', '목' ,'수', '월', '화']
index = np.arange(train['요일'].nunique())
alpha = 0.5
plt.figure(figsize=(12, 8))

plt.bar(index, train.groupby(['요일'])['시간외근무승인'].sum(), label='야근', width=0.4)
plt.bar(index+bar_width, train.groupby(['요일'])['석식계'].sum(), label='석식', width=0.4)
plt.xticks(index, label)
plt.legend('best', labels=['야근', '석식'])
plt.title('요일 별 야근 인원', fontsize=12)
plt.show()

### 요일 별 출장자
* 요일별로 출장자의 차이가 크지는 않음
    * 월요일보다 금요일이 조금 많은 정도
    * 하지만 출장자와 중,석식은 관련이 있음
        * 근무지내 출장자와 국내출장자는 어떻게 구분할까
            * 근무지내 출장자 : 출장에 따라 밥을 먹을수도 아닐수도
            * 국내 출장자 : 밥 먹는게 불가능

In [ ]:
plt.bar(train.groupby(['요일'])['출장자'].sum().index, train.groupby(['요일'])['출장자'].sum())
plt.title('요일 별 출장 인원', fontsize=12)
plt.show()

### 요일 별 휴가자
* 월요일과 금요일이 훨씬 많음
    * 휴가는 붙여서 사용 가능
        * 당일 사용 가능
    * 공휴일과도 연관성이 짙을 것
    * 날씨와도 연관성이 있지 않을까

In [ ]:
plt.bar(train.groupby(['요일'])['휴가자'].sum().index, train.groupby(['요일'])['휴가자'].sum())
plt.title('요일 별 휴가 인원', fontsize=12)
plt.show()

In [ ]:
train['년'] = train['일자'].dt.year
train['월'] = train['일자'].dt.month
train['일'] = train['일자'].dt.day

test['년'] = test['일자'].dt.year
test['월'] = test['일자'].dt.month
test['일'] = test['일자'].dt.day

### 재택 근무자 추이
* 코로나 검사와 높은 연관성이 있는 것으로 추정
    * 검사를 받을 경우 재택이라고 함

In [ ]:
plt.plot(train['일자'].iloc[800:], train['재택근무자'].iloc[800:])
plt.plot(train['일자'].iloc[800:], train['중식계'].iloc[800:])
plt.xticks(rotation=20)
plt.title('재택 근무자 추이')

plt.show()

In [ ]:
train['월일'] = train['일자'].apply(lambda x : str(x)[5:10])
train_18 = train.loc[train['년']==2018]
train_19 = train.loc[train['년']==2019]
train_20 = train.loc[train['년']==2020]

In [ ]:
plt.plot(train.groupby(['년'])['정원'].mean())
plt.title('년도 별 정원', fontsize=12)

plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
bar_width = 0.3
index = train_19.groupby(['월'])['중식계'].sum().index
alpha = 0.5

plt.bar(index-bar_width, train_18.groupby(['월'])['중식계'].sum(),
        label='2018', width=0.3, alpha=alpha)
plt.bar(index, train_19.groupby(['월'])['중식계'].sum(), label='2019', width=0.3, alpha=alpha)
plt.bar(index+bar_width,
        train_20.groupby(['월'])['중식계'].sum(), label='2020', width=0.3, alpha=alpha)
plt.bar(index+bar_width,
        train_20.groupby(['월'])['재택근무자'].sum(), label='재택근무자',
        bottom=train_20.groupby(['월'])['중식계'].sum(), width=0.3, alpha=alpha)

plt.xticks(index, label=index)
plt.xlabel('월')
plt.ylabel('중식계 총합')
plt.legend('best', labels=['2018', '2019', '2020', '재택근무자'])
plt.title('코로나가 중식계에 끼친 영향', fontsize=12)
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
bar_width = 0.3
index = train_19.groupby(['월'])['석식계'].sum().index
alpha = 0.5

plt.bar(index-bar_width, train_18.groupby(['월'])['석식계'].sum(),
        label='2018', width=0.3, alpha=alpha)
plt.bar(index, train_19.groupby(['월'])['석식계'].sum(), label='2019', width=0.3, alpha=alpha)
plt.bar(index+bar_width,
        train_20.groupby(['월'])['석식계'].sum(), label='2020', width=0.3, alpha=alpha)
plt.bar(index+bar_width,
        train_20.groupby(['월'])['재택근무자'].sum(), label='재택근무자',
        bottom=train_20.groupby(['월'])['석식계'].sum(), width=0.3, alpha=alpha)

plt.xticks(index, label=index)
plt.xlabel('월')
plt.ylabel('석식계 총합')
plt.legend('best', labels=['2018', '2019', '2020', '재택근무자'])
plt.title('코로나가 석식계에 끼친 영향', fontsize=12)
plt.show()